##### Fernando Amaral
##### MaxAbsScaler 

In [ ]:
# Standardizes data between -1 and 1, without centralizing

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("standardscaler").getOrCreate()

In [2]:
from pyspark.ml.feature import MaxAbsScaler

In [3]:
carros = spark.read.csv("Carros.csv", header=True, inferSchema=True, sep=";")
carros.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [4]:
# create a new column with features vectorized
from pyspark.ml.feature import VectorAssembler
vecassembler = VectorAssembler(inputCols=["Consumo","Cilindros","Cilindradas"], outputCol="vetor")
carros_vetor = vecassembler.transform(carros)
carros_vetor.select("Consumo","Cilindros","Cilindradas","vetor").show()

+-------+---------+-----------+------------------+
|Consumo|Cilindros|Cilindradas|             vetor|
+-------+---------+-----------+------------------+
|     21|        6|        160|  [21.0,6.0,160.0]|
|     21|        6|        160|  [21.0,6.0,160.0]|
|    228|        4|        108| [228.0,4.0,108.0]|
|    214|        6|        258| [214.0,6.0,258.0]|
|    187|        8|        360| [187.0,8.0,360.0]|
|    181|        6|        225| [181.0,6.0,225.0]|
|    143|        8|        360| [143.0,8.0,360.0]|
|    244|        4|       1467|[244.0,4.0,1467.0]|
|    228|        4|       1408|[228.0,4.0,1408.0]|
|    192|        6|       1676|[192.0,6.0,1676.0]|
|    178|        6|       1676|[178.0,6.0,1676.0]|
|    164|        8|       2758|[164.0,8.0,2758.0]|
|    173|        8|       2758|[173.0,8.0,2758.0]|
|    152|        8|       2758|[152.0,8.0,2758.0]|
|    104|        8|        472| [104.0,8.0,472.0]|
|    104|        8|        460| [104.0,8.0,460.0]|
|    147|        8|        440|

In [6]:
escala = MaxAbsScaler(inputCol="vetor", outputCol="scaled")
modelo = escala.fit(carros_vetor)
carrosabs = modelo.transform(carros_vetor)
carrosabs.select("vetor","scaled").show(truncate=False)

+------------------+-----------------------------------------------+
|vetor             |scaled                                         |
+------------------+-----------------------------------------------+
|[21.0,6.0,160.0]  |[0.061946902654867256,0.75,0.05801305293691081]|
|[21.0,6.0,160.0]  |[0.061946902654867256,0.75,0.05801305293691081]|
|[228.0,4.0,108.0] |[0.6725663716814159,0.5,0.0391588107324148]    |
|[214.0,6.0,258.0] |[0.6312684365781711,0.75,0.09354604786076867]  |
|[187.0,8.0,360.0] |[0.551622418879056,1.0,0.13052936910804933]    |
|[181.0,6.0,225.0] |[0.5339233038348082,0.75,0.08158085569253082]  |
|[143.0,8.0,360.0] |[0.4218289085545723,1.0,0.13052936910804933]   |
|[244.0,4.0,1467.0]|[0.71976401179941,0.5,0.5319071791153009]      |
|[228.0,4.0,1408.0]|[0.6725663716814159,0.5,0.5105148658448151]    |
|[192.0,6.0,1676.0]|[0.5663716814159292,0.75,0.6076867295141407]   |
|[178.0,6.0,1676.0]|[0.5250737463126843,0.75,0.6076867295141407]   |
|[164.0,8.0,2758.0]|[0.48377581120